# Module 3 - TensorFlow Mechanics 101

This is step-by-step follow along based on Google's Tensorflow Tutorial.

[TensorFlow Mechanics 101](https://www.tensorflow.org/get_started/mnist/mechanics)

The tutorial is structured to go over python codes which was supposed to be run from the command line:

python fully_connected_feed.py

There are many Python and TensorFlow programming concepts which I want to explore in greater detail. So I am re-doing this section in Jupyter Notebook.

This first section deals with Python's argparse module. It is important to master this because it will enable me to write userfriendly command-line interfaces for Python scripts.


## SECTION 1 - argparse

The argparse module is used to write user-friendly command-line interfaces. 

The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. 

The argparse module automatically generates help and usage messages and issues errors when users give the program invalid arguments.

In [ ]:
"""Functions for downloading and reading MNIST data."""

# These 3 lines provides backward compatibility with older Python versions from Python 3 code
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# six is a package that helps in writing code that is compatible with both Python 2 and Python 3.
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin

import gzip
import os
import sys
import time
import math
import tempfile
import argparse
import numpy
import tensorflow as tf

# The mnist read_data_sets() function will be used in full_connected_feed.py to download mnist dataset
# to your local training folder and to then unpack that data to return a dictionary of DataSet instances.
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

In [ ]:
import argparse

# The first step in using the argparse is creating an ArgumentParser object, which will hold 
# all the information necessary to parse the command line into Python data types.
parser = argparse.ArgumentParser(description='Process some integers.')

In [ ]:
# these calls tell the ArgumentParser how to take the strings on the command line and turn them into objects.
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

In [ ]:
# parse_args() builds a Namespace object from attributes parsed out of the command line

parser.parse_args(['--sum', '7', '-1', '42'])

## Printing out the Help Message

Assuming the Python code above is saved into a file called prog.py, it can be run at the command line and provides useful help messages:

```sh
python prog.py -h
usage: prog.py [-h] [--sum] N [N ...]

Process some integers.

positional arguments:
 N           an integer for the accumulator

optional arguments:
 -h, --help  show this help message and exit
 --sum       sum the integers (default: find the max)
```

In [ ]:
parser.print_help()

## add_argument() method

The add_argument() method must know whether an optional argument, like -f or --foo, or a positional argument, like a list of filenames, is expected. 

When parse_args() is called, optional arguments will be identified by the - prefix, and the remaining arguments will be assumed to be positional.

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-f', '--foo')
parser.add_argument('bar')

In [ ]:
parser.parse_args(['BAR'])

In [ ]:
parser.parse_args(['BAR', '--foo', 'FOO'])

In [ ]:
parser.parse_args(['--foo', 'FOO'])

## action

The action keyword argument specifies how the command-line arguments should be handled.

In [ ]:
parser = argparse.ArgumentParser()

# action='store_const' - stores the value specified by the const keyword argument
parser.add_argument('--foo', action='store_const', const=42)
parser.parse_args(['--foo'])


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--bar', action='store_false')
parser.add_argument('--baz', action='store_false')  # If not called, baz= True
parser.parse_args('--foo --bar'.split())

## nargs

nargs is more complicated.

nargs=N (where N is a number) is rather intuitive!!!

In [ ]:
parser = argparse.ArgumentParser()

# nargs=N - N arguments from the command line will be gathered together into a list
parser.add_argument('--foo', nargs=2)
parser.add_argument('bar', nargs=1)
parser.parse_args('c --foo a b'.split())

## nargs='?'


1. One argument will be consumed from the command line if possible, and produced as a single item. 

2. If no command-line argument is present, the value from default will be produced. 

3. For optional arguments, there is an additional case - the option string is present but not followed by a command-line argument. In this case the value from const will be produced.

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--foo', nargs='?', const='c', default='d')
parser.add_argument('bar', nargs='?', default='d')

In [ ]:
# One argument will be consumed from the command line if possible, and produced as a single item
parser.parse_args(['XX', '--foo', 'YY'])

In [ ]:
# For optional arguments, if the option string is present but not followed by a command-line argument,
# then the value from const will be produced.
parser.parse_args(['XX', '--foo'])

In [ ]:
# If no command-line argument is present, the value from default will be produced.
parser.parse_args([])

## Optional input and output files

A common use of nargs='?' is to allow optional input and output files:

In [ ]:
import sys

parser = argparse.ArgumentParser()
parser.add_argument('infile', nargs='?', type=argparse.FileType('r'),
                     default=sys.stdin)
parser.add_argument('outfile', nargs='?', type=argparse.FileType('w'),
                     default=sys.stdout)
parser.parse_args(['mechanics/input.txt', 'mechanics/output.txt'])

In [ ]:
parser.parse_args([])

## nargs='*' or '+'

All command-line arguments present are gathered into a list.

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('foo', nargs='+')
parser.parse_args(['a', 'b'])

## default

The default keyword argument of add_argument(), whose value defaults to None, specifies what value should be used if the command-line argument is not present. 

For optional arguments, the default value is used when the option string was not present at the command line

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', default=42)
parser.parse_args(['--foo', '2'])

In [ ]:
parser.parse_args([])

## type

The type keyword argument of add_argument() allows any necessary type-checking and type conversions to be performed.

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--height', default='10')
# the parser applies any type conversion argument before setting the attribute on the Namespace return value
parser.add_argument('--length', default='10', type = int) 
parser.add_argument('--width', default=10.5, type = int)
parser.parse_args([])

## type=callable

type can take any callable that takes a single string argument and returns the converted value. This is a neat trick!!!

In [ ]:
import math

def perfect_square(string):
    value = int(string)
    sqrt = math.sqrt(value)
    if sqrt != int(sqrt):
        msg = "%r is not a perfect square" % string
        raise argparse.ArgumentTypeError(msg)
    return value

parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('foo', type=perfect_square)
parser.parse_args(['9'])

In [ ]:
parser.parse_args(['18'])

## choice

The choices keyword argument can be used to check against a range of values:

In [ ]:
parser = argparse.ArgumentParser(prog='game.py')
parser.add_argument('move', choices=['rock', 'paper', 'scissors'])
parser.parse_args(['rock'])


In [ ]:
parser.parse_args(['fire'])

In [ ]:
parser = argparse.ArgumentParser(prog='doors.py')
parser.add_argument('door', type=int, choices=range(1, 4))
print(parser.parse_args(['3']))


In [ ]:
print(parser.parse_args(['4']))

## required

In general, the argparse module assumes that flags like -f and --bar indicate optional arguments.

To make an option required, True can be specified for the required= keyword argument to add_argument()

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', required=True)
parser.parse_args([])

## help

When a user requests help (usually by using -h or --help at the command line), these help descriptions will be displayed with each argument.

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true',
                     help='foo the bars before frobbling')
parser.add_argument('bar', nargs='+',
                     help='one of the bars to be frobbled')
parser.parse_args(['BAR', 'NONe'])

In [ ]:
parser.parse_args(['--h'])

## parse_args() method

Convert argument strings to objects and assign them as attributes of the namespace. Return the populated namespace. By default, the argument strings are taken from sys.argv, and a new empty Namespace object is created for the attributes.

The parse_args() method supports several ways of specifying the value of an option:

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x')
parser.add_argument('--foo')
parser.parse_args(['-x', 'X'])

In [ ]:
parser.parse_args(['--foo=FOO'])

## namespace=object
It may also be useful to have an ArgumentParser assign attributes to an already existing object, rather than a new Namespace object. This can be achieved by specifying the namespace= keyword argument.

A rather neat trick!!!

In [ ]:
class C(object):
    pass

c = C()
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
parser.parse_args(args=['--foo', 'BAR'], namespace=c)

# This is rather cool!!!!
c.foo

## fully_connected_feed.py

The following code implements the command-line interface for fully_connected_feed.py.

In [ ]:
parser = argparse.ArgumentParser()
    
parser.add_argument(
      '--learning_rate',
      type=float,
      default=0.01,
      help='Initial learning rate.'
)

parser.add_argument(
      '--max_steps',
      type=int,
      default=10000,
      help='Number of steps to run trainer.'
)
    
parser.add_argument(
      '--hidden1',
      type=int,
      default=128,
      help='Number of units in hidden layer 1.'
)

parser.add_argument(
      '--hidden2',
      type=int,
      default=32,
      help='Number of units in hidden layer 2.'
)
    
parser.add_argument(
      '--batch_size',
      type=int,
      default=100,
      help='Batch size.  Must divide evenly into the dataset sizes.'
)

parser.add_argument(
      '--input_data_dir',
      type=str,
      default='/tmp/tensorflow/mnist/input_data',
      help='Directory to put the input data.'
)
    
parser.add_argument(
      '--log_dir',
      type=str,
      default='/tmp/tensorflow/mnist/logs/fully_connected_feed',
      help='Directory to put the log data.'
)

parser.add_argument(
      '--fake_data',
      default=False,
      help='If true, uses fake data for unit testing.',
      action='store_true'
)




In [ ]:
parser.print_help()

In [ ]:
# Sometimes a script may only parse a few of the command-line arguments, passing the remaining arguments on to another 
# script or program. parse_known_args() returns a two item tuple containing the populated namespace (into FLAG) and the
# list of remaining argument strings.
FLAGS, unparsed = parser.parse_known_args(['--max_steps','2000'])

# FLAGS is the Namespace which stores all the parameters

FLAGS

In [ ]:
unparsed

## Implemented in command_line.py

It works as advertised!!!

```sh
$ python ./mechanics/command_line.py --help 
usage: command_line.py [-h] [--learning_rate LEARNING_RATE]
                       [--max_steps MAX_STEPS] [--hidden1 HIDDEN1]
                       [--hidden2 HIDDEN2] [--batch_size BATCH_SIZE]
                       [--input_data_dir INPUT_DATA_DIR] [--log_dir LOG_DIR]
                       [--fake_data]

optional arguments:
  -h, --help            show this help message and exit
  --learning_rate LEARNING_RATE
                        Initial learning rate.
  --max_steps MAX_STEPS
                        Number of steps to run trainer.
  --hidden1 HIDDEN1     Number of units in hidden layer 1.
  --hidden2 HIDDEN2     Number of units in hidden layer 2.
  --batch_size BATCH_SIZE
                        Batch size. Must divide evenly into the dataset sizes.
  --input_data_dir INPUT_DATA_DIR
                        Directory to put the input data.
  --log_dir LOG_DIR     Directory to put the log data.
  --fake_data           If true, uses fake data for unit testing.
```

```sh
$ python3 ./mechanics/command_line.py
This script is being run directly
There is no code yet in this python file!!!
```